# Conditional Independence

Let us now work through an examples of conditional independence with another examples and some accompanying calculations.

Before doing so, it is useful to check out a couple of properties of the factorisation of joint distributions. Consider the following joint distrubution of two variables:

|       | $Y=0$ | $Y=1$ |
|:-----:|:-----:|:-----:|
| $X=0$ | 0.4   | 0.3   | 
| $X=1$ | 0.2   | 0.1   |

We can calculate the univariate distributions $P(X)$ and $P(Y)$ from this using $P(X) = \sum_Y P(X,Y)$ and similarly for $P(Y)$:

$$
P(X) = \begin{pmatrix}0.7 \\ 0.3\end{pmatrix}\quad\mathrm{and}\quad P(Y) = \begin{pmatrix}0.6 \\ 0.4\end{pmatrix}
$$

Let us now assume that all we have is $P(X)$ and $P(Y)$ and we want the joint distribution. By assuming that $X$ and $Y$ are independent we can calculate $P(X,Y)=P(X)P(Y)$:

|       | $Y=0$ | $Y=1$ |
|:-----:|:-----:|:-----:|
| $X=0$ | 0.42  | 0.28  | 
| $X=1$ | 0.18  | 0.12  |

This is close to the true joint distribution, but not the same. We will now perform a similar analysis on a somewhat more complex problem to illustrate the notion of statistical independence.

A survey was done of school-leavers' employment ten years after they finished high school. The survey look at their careers (which sector they worked in and what type of work they did) and the subjects in which they graduated from high school. We will consider a small part of this survey data and will look at the subset of people who
* Studied Mathematics and/or Geography (or neither)
* Pursued careers in Programming and/or Finance (or neither)

In other words, the full joint distribution has been marginalised over all variables except
* $M$: studied Mathematics.
* $G$: studied Geography.
* $F$: works in finance.
* $P$: works as a programmer.

The joint distribution of these variables is shown below:

![Joint Probability Distribution of student's school subjects and their career path.](../images/jointpdf.png)

What can we deduce from this?

* It seems that both Programmers and people who work in Finance are much more likely to have studied Maths than not.
* Studying Geography does not seem to make much impact on whether someone works in Finance or as a Programmer.
* Amongst people who studied Maths, working as a Programmer outside of Finance is just as likely as working as a Programmer in Finance.

Let's try to calculate some relevant quantities. We'll first populate the joint distribution which we'll put in a 4d `numpy` array.

```
JointPDF[iG][iM][iF][iP]
```

where `{iG,iM,iF,iP} = {0,1}` with 0 representing False and 1 representing True. Thus `JointPDF[1][0][0][1]` represents $P(G\land\lnot M \land \lnot F \land P)$. 

In [1]:
import numpy as np
JointPDF = np.zeros([2,2,2,2])
JointPDF[0][0][:][:] = np.array([[.066,0.054],[0.0165,0.0135]])
JointPDF[0][1][:][:] = np.array([[.081,.099],[.1215,.1485]])
JointPDF[1][0][:][:] = np.array([[.044,.036],[.011,.009]])
JointPDF[1][1][:][:] = np.array([[.054,.066],[.081,.099]])

We can calculate quantities from this by summing over the array in different ways. For example, to calculate the univariate distributions we sum over the other indices:

In [2]:
# Check that the total probability sums to 1
P = JointPDF.sum(axis=(0,1,2,3))
print(f"Total probability = {P}")

# Probability of studying/not studying German
P_G = JointPDF.sum(axis=(1,2,3))
print(f"P_G={P_G}")

# Probability of studying/not studying Math
P_M = JointPDF.sum(axis=(0,2,3))
print(f"P_M={P_M}")

# Probability of working in finance
P_F = JointPDF.sum(axis=(0,1,3))
print(f"P_F={P_F}")

# Probability of working as a programmer
P_P = JointPDF.sum(axis=(0,1,2))
print(f"P_P={P_P}")

# Joint probability of working in finance and working as a programmer
JointPDF.sum(axis=(0,1))


Total probability = 1.0
P_G=[0.6 0.4]
P_M=[0.25 0.75]
P_F=[0.5 0.5]
P_P=[0.475 0.525]


array([[0.245, 0.255],
       [0.23 , 0.27 ]])

Let us first explore whether the four variables are independent of each other. To test this, we see if elements of the joint distribution are given by the products of the univariate distributions: $P(G,M,F,P)=P(G)P(M)P(F),P(P)$.

In [5]:
import itertools
# This iterator creates all combinations of the indices.
iterator = itertools.product((0,1),repeat=4)
IndependentPDF = np.zeros([2,2,2,2])
for i in iterator:
    IndependentPDF[i[0],i[1],i[2],i[3]] = P_G[i[0]] *  P_M[i[1]] * P_F[i[2]] * P_P[i[3]]

print(f"Independent PDF: {IndependentPDF}")
P = JointPDF.sum(axis=(0,1,2,3))


Independent PDF: [[[[0.035625 0.039375]
   [0.035625 0.039375]]

  [[0.106875 0.118125]
   [0.106875 0.118125]]]


 [[[0.02375  0.02625 ]
   [0.02375  0.02625 ]]

  [[0.07125  0.07875 ]
   [0.07125  0.07875 ]]]]


This is clearly not the same as the known joint PDF. Our variables are clearly not independent of each other.

Let's dig a little deeper and compute a few more things. This time, we'll focus on computing some conditional distributions. We'll focus on computing distributions that are conditional on what people studied - that seems more intuitively interesting.

First, let's compute the distributions of $M$, $F$, and $P$ conditioned on $G$

In [6]:
P_M_G = JointPDF.sum(axis=(2,3))[0]/P_G[0]
P_M_notG = JointPDF.sum(axis=(2,3))[1]/P_G[1]
P_F_G = JointPDF.sum(axis=(1,3))[0]/P_G[0]
P_F_notG = JointPDF.sum(axis=(1,3))[1]/P_G[1]
P_P_G = JointPDF.sum(axis=(1,2))[0]/P_G[0]
P_P_notG = JointPDF.sum(axis=(1,2))[1]/P_G[1]

print(f"P_M_G = {P_M_G}")
print(f"P_M_notG = {P_M_notG}")
print(f"P_F_G = {P_F_G}")
print(f"P_F_notG = {P_F_notG}")
print(f"P_P_G = {P_P_G}")
print(f"P_P_notG = {P_P_notG}")

P_M_G = [0.25 0.75]
P_M_notG = [0.25 0.75]
P_F_G = [0.5 0.5]
P_F_notG = [0.5 0.5]
P_P_G = [0.475 0.525]
P_P_notG = [0.475 0.525]


Interesting. Each of the other variables are **independent** of $G$. Perhaps we might have expected this? Would we expect whether someone has studied Geography or not to influence whether they also studies Maths, or work in finance, or work as a programmer. What about studying Maths? We might expect that this would influence whether someone works as a programmer or in finance?

In [77]:

P_F_M = JointPDF.sum(axis=(0,3))[0]/P_M[0]
P_F_notM = JointPDF.sum(axis=(0,3))[1]/P_M[1]
P_P_M = JointPDF.sum(axis=(0,2))[0]/P_M[0]
P_P_notM = JointPDF.sum(axis=(0,2))[1]/P_M[1]


print(f"P_F_M = {P_F_M}")
print(f"P_F_notM = {P_F_notM}")
print(f"P_P_M = {P_P_M}")
print(f"P_P_notM = {P_P_notM}")

P_F_M = [0.8 0.2]
P_F_notM = [0.4 0.6]
P_P_M = [0.55 0.45]
P_P_notM = [0.45 0.55]


Indeed it does. We see that those who studied Maths are much more likely to work in finance than those who didn't, and a bit more likely to work in programming. $P$ and $F$ are thus independent of $G$ and are conditionally dependent on $M$.

We might also look at their joint distribution:

In [82]:
P_FP_M = JointPDF.sum(axis=(0))[0]/P_M[0]
P_FP_notM = JointPDF.sum(axis=(0))[1]/P_M[1]
print(f"P_FP_M = {P_FP_M}")
print(f"P_FP_notM = {P_FP_notM}")

P_FP_M = [[0.44 0.36]
 [0.11 0.09]]
P_FP_notM = [[0.18 0.22]
 [0.27 0.33]]


What can we see here? Nothing, at first. But let's look more closely. We see that $P(F\land P\vert M) = 0.44 = 0.8\times 0.55 = P(F\vert M)P(P\vert M)$. The same is true of the other terms in the Joint PDF. 

We therefore have that $P(F,P\vert M) = P(F\vert M)P(P\vert M)$, $F$ and $P$ are said to be **conditionally independent* of each other. 

So what do we now know?

* $G$ is independent of the other variables.
* $P$ and $F$ depend on $M$
* $P$ and $F$ are independent of each other, given $M$.


We have two questions: does this make intuitive sense, and what does it mean mathematically?

Certainly it makes intuitive sense. We would probably not expect study of German to be predictive of future careers in finance or programming, nor would we expect it to be related to the study of Maths (we would probably have seen a different set of results if we had considered Physics instead of German). It also seems reasonable that careers in programming or finance would depend on a previous study of mathematics since we know that mathematical skills are important in both of these areas. What about our finding that careers in finance and programming are independent of each other given previous study of mathematics? Yes, this seems reasonable too. Many programmers work in finance, but many also work in other industries. We conclude, therefore that these findings make sense as they seem to capture the natural relationships between the different factors.

What about the mathematical implications? Let us recall the product rule of probability

$P(X_i,X_j) = P(X_i\vert X_j)P(X_j)$

For larger numbers of variables, this generalises to 

$P(X_1,X_2,\dots X_N) = P(X_N\vert X_1,\dots,X_{N-1})P(\vert X_1,\dots,X_{N-1})$

We can further write $P(\vert X_1,\dots,X_{N-1}) = P(X_{N-1}\vert X_1,\dots,X_{N-2})P(\vert X_{N-1},\dots,X_{N-2})$. This can be repeated until we end up with

$P(X_1,X_2,\dots X_N) = P(X_N\vert X_1,\dots,X_{N-1})P(X_{N-1}\vert X_1,\dots,X_{N-2})P(\vert X_{N-1},\dots,X_{N-2})\cdots P(X_3\vert X_2,X_1)P(X_2\vert X_1)P(X_1)$

This means that we can write any joint distribution as a product of conditional distributions.

For our current problem, we can identify the following distributions that are meaningful:
* $P(G)$
* $P(M)$
* $P(F\vert M)$
* $P(P\vert M)$

and can therefore factorise the joint distribution as

$P(G,M,F,P) = P(P\vert M)P(F\vert M)P(M)P(G)$

The factorisation of the joint distribution via independence and conditional independence is central to the concept of the **Bayesian Network** which will be our next topic.


